In [1]:
!pip3 uninstall evo -y
!python3 -m pip install -e /home/joshuabird/Documents/multi-agent-evo

Found existing installation: evo 1.26.1
Uninstalling evo-1.26.1:
  Successfully uninstalled evo-1.26.1
Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///home/joshuabird/Documents/multi-agent-evo
  Preparing metadata (setup.py) ... done
  Running setup.py develop for evo


In [59]:
from evo.tools import log
log.configure_logging()
from evo.tools import plot
from evo.tools.plot import PlotMode
from evo.core.metrics import PoseRelation
from evo.core.units import Unit
from evo.tools.settings import SETTINGS

# temporarily override some package settings
SETTINGS.plot_figsize = [6, 6]
SETTINGS.plot_split = True
SETTINGS.plot_usetex = False

# magic plot configuration
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib widget

# interactive widgets configuration
import ipywidgets

check_opts_ape = {"align": False, "correct_scale": False, "show_plot": True}
check_boxes_ape=[ipywidgets.Checkbox(description=desc, value=val) for desc, val in check_opts_ape.items()]
check_opts_rpe = {"align": False, "correct_scale": False, "all_pairs": False, "show_plot": True}
check_boxes_rpe=[ipywidgets.Checkbox(description=desc, value=val) for desc, val in check_opts_rpe.items()]
delta_input = ipywidgets.FloatText(value=1.0, description='delta', disabled=False, color='black')
delta_unit_selector=ipywidgets.Dropdown(
    options={u.value: u for u in Unit if u is not Unit.seconds},
    value=Unit.frames, description='delta_unit'
)
plotmode_selector=ipywidgets.Dropdown(
    options={p.value: p for p in PlotMode},
    value=PlotMode.xy, description='plot_mode'
)
pose_relation_selector=ipywidgets.Dropdown(
    options={p.value: p for p in PoseRelation},
    value=PoseRelation.translation_part, description='pose_relation'
)

agent_ids = [1,2,3]

In [60]:
from evo.tools import file_interface
from rosbags.rosbag2 import Reader as Rosbag2Reader
from rosbags.serde import deserialize_cdr
import math
import numpy as np
from scipy.spatial.transform import Rotation
from pathlib import Path
import os
from rosbags.typesys import get_types_from_idl, get_types_from_msg, register_types
import copy

# Add custom types
interfaces_dir = "../interfaces/msg/"
for file in os.listdir(interfaces_dir):
    msg_text = Path(interfaces_dir + file).read_text()
    register_types(get_types_from_msg(msg_text, "interfaces/msg/"+file[:-4]))

bag_name = "/home/joshuabird/Desktop/Apr7-MH-trajectorya"

traj_ests = {}
traj_refs = {}
with Rosbag2Reader(bag_name) as reader:
    # Read the trajectory and ground truth for each agent
    for agent_id in agent_ids:
        traj_est = file_interface.read_bag_trajectory(reader, f"/robot{agent_id}/camera_pose", transform_to_root_tf_frame_id="/world", use_final_tf_state=True)
        traj_ref = file_interface.read_bag_trajectory(reader, f"/robot{agent_id}/ground_truth_pose")

        traj_est.timestamps += 0.17

        traj_ests[agent_id] = traj_est
        traj_refs[agent_id] = traj_ref

[WARNING] geometry_msgs/PointStamped does not contain rotation, evo will use unit quaternion. Note that rotation metrics will be invalid and RPE will only be valid with point_distance metric.
[WARNING] geometry_msgs/PointStamped does not contain rotation, evo will use unit quaternion. Note that rotation metrics will be invalid and RPE will only be valid with point_distance metric.
[WARNING] geometry_msgs/PointStamped does not contain rotation, evo will use unit quaternion. Note that rotation metrics will be invalid and RPE will only be valid with point_distance metric.


In [61]:
from evo.core.trajectory import PosePath3DElement

filter_func: PosePath3DElement = lambda element: element["coord_frame"] == "robot1/origin"

# Filter to only the time range where trajectories are merged
for agent_id in agent_ids:
    traj_ests[agent_id].filter(filter_func)

# Trim them down to the shortest time range
# max_start_time = max([min(traj_ests[agent_id].timestamps) for agent_id in agent_ids] + [min(traj_refs[agent_id].timestamps) for agent_id in agent_ids])
# min_end_time = min([max(traj_ests[agent_id].timestamps) for agent_id in agent_ids] + [max(traj_refs[agent_id].timestamps) for agent_id in agent_ids])

# for agent_id in agent_ids:
#     traj_ests[agent_id].reduce_to_time_range(max_start_time, min_end_time)
#     traj_refs[agent_id].reduce_to_time_range(max_start_time, min_end_time)

In [62]:
for agent_id in agent_ids:
    print(traj_ests[agent_id])
    print(traj_refs[agent_id])

3619 poses, 17.055m path length, 181.200s duration
3058 poses, 79.241m path length, 183.710s duration
3002 poses, 15.367m path length, 150.150s duration
2428 poses, 72.737m path length, 149.610s duration
2620 poses, 26.578m path length, 131.100s duration
1858 poses, 126.892m path length, 132.256s duration


In [63]:
from evo.core import sync
from evo.core.trajectory import align_multiple

max_diff = 0.5

for agent_id in agent_ids:
    traj_refs[agent_id], traj_ests[agent_id] = sync.associate_trajectories(traj_refs[agent_id], traj_ests[agent_id], max_diff)
    
align_multiple(list(traj_ests.values()), list(traj_refs.values()), correct_scale=True)

[array([[ 0.33391169, -0.89987006, -0.28060091],
        [ 0.94173459,  0.30569379,  0.14031135],
        [-0.04048403, -0.31110318,  0.94951348]]),
 array([ 4.80663452, -1.80990514,  1.0773241 ]),
 4.875148051082737]

In [64]:
for agent_id in agent_ids:
    print(traj_refs[agent_id])
    print(traj_ests[agent_id])
    print()

3020 poses, 79.240m path length, 181.510s duration
3020 poses, 82.137m path length, 181.000s duration

2428 poses, 72.737m path length, 149.610s duration
2428 poses, 74.477m path length, 149.450s duration

1853 poses, 126.891m path length, 131.955s duration
1853 poses, 126.341m path length, 131.100s duration



In [65]:
import evo.main_ape as main_ape
import evo.common_ape_rpe as common

count = 0

def callback_ape(pose_relation, align, correct_scale, plot_mode, show_plot):
    global results, count
    est_name="APE Test #{}".format(count)
    
    fig = plt.figure()
    ax = plot.prepare_axis(fig, plot_mode)
    
    overall_min = np.inf
    overall_max = -np.inf
    
    results = {}

    for agent_id in agent_ids:
        result = main_ape.ape(traj_refs[agent_id], traj_ests[agent_id], est_name=est_name,
                              pose_relation=pose_relation, align=align, correct_scale=correct_scale)
        results[agent_id] = result

        if result.stats["min"] < overall_min:
            overall_min = result.stats["min"]
        if result.stats["max"] > overall_max:
            overall_max = result.stats["max"]

    for agent_id in agent_ids:
        if show_plot:
            result = results[agent_id]

            plot.traj(ax, plot_mode, traj_refs[agent_id], style="--", alpha=0.5)
            plot.traj_colormap(
                ax, result.trajectories[est_name], result.np_arrays["error_array"], plot_mode,
                min_map=overall_min, max_map=overall_max, color_bar_label="APE [m]", show_color_bar=agent_id == agent_ids[-1])
    count += 1
    
    
    
_ = ipywidgets.interact_manual(callback_ape, pose_relation=pose_relation_selector, plot_mode=plotmode_selector,
                               **{c.description: c.value for c in check_boxes_ape})

interactive(children=(Dropdown(description='pose_relation', index=1, options={'full transformation': <PoseRela…

In [66]:
import evo.main_ape as main_ape
import evo.common_ape_rpe as common

count = 0

def callback_ape(pose_relation, align, correct_scale, show_plot):
    global results, count
    est_name="APE Test #{}".format(count)
    
    fig, axs = plt.subplots(len(agent_ids), 2, sharex=False, sharey=True,gridspec_kw={'width_ratios': [5, 1]}, figsize=(10, 3*len(agent_ids)))
    plt.subplots_adjust(hspace=0.1, wspace=0.02)
    fig.suptitle("APE vs. Time")

    # share x axis along first columns
    if len(agent_ids) > 1:
        axs[0][0].get_shared_x_axes().join(*[axs[agent_id-agent_ids[0]][0] for agent_id in agent_ids])
    
    overall_min = np.inf
    overall_max = -np.inf
    
    results = {}

    for agent_id in agent_ids:
        result = main_ape.ape(traj_refs[agent_id], traj_ests[agent_id], est_name=est_name,
                              pose_relation=pose_relation, align=align, correct_scale=correct_scale)
        results[agent_id] = result

        if result.stats["min"] < overall_min:
            overall_min = result.stats["min"]
        if result.stats["max"] > overall_max:
            overall_max = result.stats["max"]

    for agent_id in agent_ids:
        if show_plot:
            result = results[agent_id]
            if len(agent_ids) > 1:
                ax_row = axs[agent_id-agent_ids[0]]
            else:
                ax_row = axs

            plot.error_array(ax_row[0], result.np_arrays["error_array"], x_array=result.np_arrays["seconds_from_start"], color=None, xlabel="Time (s)" if agent_id == agent_ids[-1] else None, ylabel=f"Agent {agent_id}\nAPE [m]")
            if agent_id != agent_ids[-1]:
                ax_row[0].set_xticklabels([])

            print(np.sqrt(np.mean(result.np_arrays["error_array"]**2)))

            ax_row[1].hist(result.np_arrays["error_array"], bins=30, orientation="horizontal", edgecolor='none')
            ax_row[1].set_xticklabels([])


    count += 1
    
    
    
_ = ipywidgets.interact_manual(callback_ape, pose_relation=pose_relation_selector,
                               **{c.description: c.value for c in check_boxes_ape})

interactive(children=(Dropdown(description='pose_relation', index=1, options={'full transformation': <PoseRela…

In [67]:
import evo.main_rpe as main_rpe

count = 0

def callback_rpe(pose_relation, delta, delta_unit, all_pairs, align, correct_scale, plot_mode, show_plot):
    global results, count
    est_name="APE Test #{}".format(count)
    
    fig = plt.figure()
    ax = plot.prepare_axis(fig, plot_mode)
    
    overall_min = np.inf
    overall_max = -np.inf
    
    results = {}

    for agent_id in agent_ids:
        result = main_rpe.rpe(traj_refs[agent_id], traj_ests[agent_id], est_name=est_name,
                            pose_relation=pose_relation, delta=delta, delta_unit=delta_unit, 
                            all_pairs=all_pairs, align=align, correct_scale=correct_scale, 
                            support_loop=True)
        results[agent_id] = result

        if result.stats["min"] < overall_min:
            overall_min = result.stats["min"]
        if result.stats["max"] > overall_max:
            overall_max = result.stats["max"]

    for agent_id in agent_ids:
        if show_plot:
            result = results[agent_id]

            plot.traj(ax, plot_mode, traj_refs[agent_id], style="--", alpha=0.5)
            plot.traj_colormap(
                ax, result.trajectories[est_name], result.np_arrays["error_array"], plot_mode,
                min_map=overall_min, max_map=overall_max, color_bar_label="RPE [m]", show_color_bar=agent_id == agent_ids[-1])
    count += 1
    
    
    
_ = ipywidgets.interact_manual(callback_rpe, pose_relation=pose_relation_selector, plot_mode=plotmode_selector, 
                               delta=delta_input, delta_unit=delta_unit_selector, 
                               **{c.description: c.value for c in check_boxes_rpe})


interactive(children=(Dropdown(description='pose_relation', index=1, options={'full transformation': <PoseRela…

In [68]:
from rosbags.rosbag2 import Writer as Rosbag2Writer
from rclpy.serialization import serialize_message
from rosbags.typesys.types import sensor_msgs__msg__CompressedImage, sensor_msgs__msg__CameraInfo, tf2_msgs__msg__TFMessage
from cv_bridge import CvBridge
import cv2
from sensor_msgs.msg import CameraInfo
from geometry_msgs.msg import TransformStamped


with Rosbag2Writer(f"{bag_name}_processed") as writer:
    for agent_id in agent_ids:
        min_timestamp = np.min(traj_ests[agent_id].timestamps.tolist() + traj_ests[agent_id].timestamps.tolist())-1000
        traj_ests[agent_id].timestamps -= min_timestamp
        traj_refs[agent_id].timestamps -= min_timestamp

        file_interface.write_bag_trajectory(writer, traj_ests[agent_id], f"/robot{agent_id}/camera_pose", frame_id="world")
        file_interface.write_bag_trajectory(writer, traj_refs[agent_id], f"/robot{agent_id}/ground_truth_pose", frame_id="world")


Saved geometry_msgs/PoseStamped topic: /robot1/camera_pose
Saved geometry_msgs/PoseStamped topic: /robot1/ground_truth_pose
Saved geometry_msgs/PoseStamped topic: /robot2/camera_pose
Saved geometry_msgs/PoseStamped topic: /robot2/ground_truth_pose
Saved geometry_msgs/PoseStamped topic: /robot3/camera_pose
Saved geometry_msgs/PoseStamped topic: /robot3/ground_truth_pose
